<a href="https://colab.research.google.com/github/iamatul1214/System-Design-For-Machine-Learning/blob/main/API_gateway_with_rate_limiting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn

## Create Backend Services (Mock Microservices)

### We’ll simulate two backend services:

- User Service

- ML Recommendation Service

In [23]:
from fastapi import FastAPI

# User Service
user_service = FastAPI()

user_data = {
    101: {"name": "Atul", "segment": "premium"},
    102: {"name": "Anagha", "segment": "luxury"},
    103: {"name": "Aman", "segment": "premium"},
    104: {"name": "Sean", "segment": "ultra luxury"}
}

@user_service.get("/users/{user_id}")
def get_user(user_id: int):
    user_info = user_data.get(user_id)
    if not user_info:
        return {"error": "User not found"}
    return {"user_id": user_id, **user_info}


# ML Recommendation Service
ml_service = FastAPI()

recommendation_data = {
    101: ["shirt", "jeans", "watch"],
    102: ["dress", "shoes", "handbag"],
    103: ["T-shirt", "jeans", "shoes"],
    104: ["Jackets", "Bracelet", "Shorts"]
}

@ml_service.get("/recommend/{user_id}")
def recommend(user_id: int):
    recommendations = recommendation_data.get(user_id)
    if not recommendations:
        return {"error": "Recommendations not found for user"}
    return {"user_id": user_id, "recommendations": recommendations}

## API Gateway (Core Logic)

### Now we create an API Gateway that:

- Authenticates

- Rate limits

- Routes requests

In [36]:
from fastapi import FastAPI, Request, HTTPException
from fastapi.testclient import TestClient
import time

api_gateway = FastAPI()

# Fake service registry
services = {
    "users": TestClient(user_service),
    "recommend": TestClient(ml_service)
}

# Simple rate limit storage
rate_limit = {}
MAX_REQUESTS = 5
WINDOW = 30  # seconds

def check_rate_limit(client_id):
    now = time.time()
    timestamps = rate_limit.get(client_id, [])
    timestamps = [t for t in timestamps if now - t < WINDOW]

    if len(timestamps) >= MAX_REQUESTS:
        raise HTTPException(status_code=429, detail="Rate limit exceeded")

    timestamps.append(now)
    rate_limit[client_id] = timestamps


@api_gateway.middleware("http")
async def gateway_middleware(request: Request, call_next):
    # Auth check
    api_key = request.headers.get("x-api-key")
    if api_key != "secret-key":
        raise HTTPException(status_code=401, detail="Unauthorized")

    # Rate limiting
    # Extract client_id from the path (e.g., from /api/users/users/101, client_id would be '101')
    path_segments = request.url.path.split('/')
    # Assuming the user_id is always the last segment in the path part of the URL and is a digit
    client_id = "anonymous" # Default if not found
    for segment in reversed(path_segments):
        if segment.isdigit():
            client_id = segment
            break

    check_rate_limit(client_id)

    return await call_next(request)


@api_gateway.get("/api/{service}/{path:path}")
def route_request(service: str, path: str):
    if service not in services:
        raise HTTPException(status_code=404, detail="Service not found")

    backend = services[service]
    response = backend.get(f"/{path}")

    return response.json()

## Let's call API gateway (client's simulation)

####  we pass wrong secret key

In [37]:
gateway_client = TestClient(api_gateway)

# Call User Service via API Gateway
response = gateway_client.get(
    "/api/users/users/101",
    headers={"x-api-key": "secret-key_1"}
)
print(response.json())



HTTPException: 401: Unauthorized

#### we pass correct secret key

In [38]:
# Call User Service via API Gateway
response = gateway_client.get(
    "/api/users/users/104",
    headers={"x-api-key": "secret-key"}       # we pass correct secret key
)

print(response.json())

{'user_id': 104, 'name': 'Sean', 'segment': 'ultra luxury'}


## call ML service via gateway client

In [39]:
response = gateway_client.get(
    "/api/recommend/recommend/103",
    headers={"x-api-key": "secret-key"}
)

response.json()

{'user_id': 103, 'recommendations': ['T-shirt', 'jeans', 'shoes']}

## Let's trigger rate limiting

In [40]:
for i in range(5):
    r = gateway_client.get(
        "/api/users/users/103",
        headers={"x-api-key": "secret-key"}
    )
    print(i + 1, r.status_code, r.json())


1 200 {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}
2 200 {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}
3 200 {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}
4 200 {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}


HTTPException: 429: Rate limit exceeded

## Let's check other service as well, if it works

In [43]:
response = gateway_client.get(
    "/api/shoppers/shop/101",
    headers={"x-api-key": "secret-key"}
)

print(response.json())

{'detail': 'Service not found'}


## We check rate limiting for all the users by hitting service

In [48]:
for i in range(5):
    for i in range(101,105,1):
        response = gateway_client.get(
        f"/api/users/users/{i}",
        headers={"x-api-key": "secret-key"}
    )

        print(f"user service: {response.json()}")

        response = gateway_client.get(
        f"/api/recommend/recommend/{i}",
        headers={"x-api-key": "secret-key"}
    )

        print(f"ML service: {response.json()}")



user service: {'user_id': 101, 'name': 'Atul', 'segment': 'premium'}
ML service: {'user_id': 101, 'recommendations': ['shirt', 'jeans', 'watch']}
user service: {'user_id': 102, 'name': 'Anagha', 'segment': 'luxury'}
ML service: {'user_id': 102, 'recommendations': ['dress', 'shoes', 'handbag']}
user service: {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}
ML service: {'user_id': 103, 'recommendations': ['T-shirt', 'jeans', 'shoes']}
user service: {'user_id': 104, 'name': 'Sean', 'segment': 'ultra luxury'}
ML service: {'user_id': 104, 'recommendations': ['Jackets', 'Bracelet', 'Shorts']}
user service: {'user_id': 101, 'name': 'Atul', 'segment': 'premium'}
ML service: {'user_id': 101, 'recommendations': ['shirt', 'jeans', 'watch']}
user service: {'user_id': 102, 'name': 'Anagha', 'segment': 'luxury'}
ML service: {'user_id': 102, 'recommendations': ['dress', 'shoes', 'handbag']}
user service: {'user_id': 103, 'name': 'Aman', 'segment': 'premium'}
ML service: {'user_id': 103, 'recomm

HTTPException: 429: Rate limit exceeded